# openai functions

References:

- https://github.com/Azure-Samples/openai/blob/main/Basic_Samples/Functions/working_with_functions.ipynb
- https://github.com/openai/openai-cookbook/blob/main/examples/How_to_call_functions_for_knowledge_retrieval.ipynb

In [72]:
import openai
import json

with open("openai_api.json") as f:
    creds = json.load(f)

openai.api_version = creds["api_version"]
openai.api_base = creds["api_base"]
openai.api_type = creds["api_type"]
openai.api_key = creds["api_key"]

## Natural language to structured DSL

Following example shows how to convert natural language queries to a structured query using OpenAI functions.
As an example we use natural language queries to search for shoes on an online store. OpenAI functions converts these queries into search filters that can be used to call a search API and show results to the user.

In [20]:
system_prompt = """You are an AI assistant for an online shoe store that helps users find appropriate shoes.
You convert their queries into appropriate filters for the search function.
"""

functions = [
    {
        "name": "search",
        "description": "This function calls Search API of the shoe store to find top matches for user's description.",
        "parameters": {
            "type": "object",
            "properties": {
                "shoe_types": {
                    "type": "array",
                    "items": {
                        "type": "string",
                        "enum": ["running", "jogging", "trekking", "sports", "heels"]
                    },
                    "description": "The types of shoes to search for."
                },
                "sizes": {
                    "type": "array",
                    "items": {
                        "type": "string",
                        "enum": ["uk4", "uk5", "uk6", "uk7", "uk8", "uk9"]
                    },
                    "description": "The sizes of shoe to search for. Small refers to uk4 and uk5. Medium refers to uk6 and uk7. Large refers to uk8 and uk9."
                },
                "brands": {
                     "type": "array",
                    "items": {
                        "type": "string",
                        "enum": ["reebok", "puma", "adidas", "clarks london"]
                    },
                    "description": "The brands of shoes to search for."
                },
                "colors": {
                     "type": "array",
                    "items": {
                        "type": "string",
                        "enum": ["red", "green", "blue"]
                    },
                    "description": "The colors of shoes to search for."
                },
            }
        },
        "result": {
            "type": "array",
            "items": {
                "type": "string"
            }
        }
    }
]

In [27]:
def get_search_filters(chat_history):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        engine="gpt-4",
        messages=[
            {"role": "system", "content": system_prompt},
            *chat_history,
        ],
        temperature=0,
        functions=functions,
    )
    response_message = response.choices[0].message
    assert "function_call" in response_message
    search_filters = json.loads(response_message.function_call.arguments)
    
    chat_history.append({
        "role": response_message["role"],
        "function_call": {
            "name": response_message["function_call"]["name"],
            "arguments": response_message["function_call"]["arguments"],
        },
        "content": None
    })
    chat_history.append({
        "role": "function",
        "name": response_message["function_call"]["name"],
        "content": "List of 10 shoes with that description",
    })
    return search_filters, chat_history

In [30]:
# simulate a chatbot style conversation
user_queries = [
    "blue running shoes for small feet",
    "show nike and puma only",
    "include red",
    "show sports shoes instead of running"
]

chat_history = []
for query in user_queries:
    chat_history.append({"role": "user", "content": query})
    search_filters, chat_history = get_search_filters(chat_history)
    print(f"Query: {query}")
    print(f"Search Filters: {search_filters}")
    print()

Query: blue running shoes for small feet
Search Filters: {'shoe_types': ['running'], 'sizes': ['uk4', 'uk5'], 'colors': ['blue']}

Query: show nike and puma only
Search Filters: {'shoe_types': ['running'], 'sizes': ['uk4', 'uk5'], 'colors': ['blue'], 'brands': ['nike', 'puma']}

Query: include red
Search Filters: {'shoe_types': ['running'], 'sizes': ['uk4', 'uk5'], 'colors': ['blue', 'red'], 'brands': ['nike', 'puma']}

Query: show sports shoes instead of running
Search Filters: {'shoe_types': ['sports'], 'sizes': ['uk4', 'uk5'], 'colors': ['blue', 'red'], 'brands': ['nike', 'puma']}



## Chatbot performing API actions

Sample chatbot created using openai functions. Example chatbot below works for an ecommerce app and helps customers solve order issues by performing actions by calling APIs. It also does information retrieval. Function schema is automatically derived from function signature and doc.

In [104]:
from importlib import reload
import functions_code
reload(functions_code)

order_api = functions_code.OrderAPI()
functions = order_api.get_functions()  # automatically generated functions from methods of OrderAPI class

In [105]:
system_prompt = """You are an AI assistant for an ecommerce app helping customers with their queries about their orders.
If customer's query requires an order id but none is present in previous messages then call list orders function with to_get_order_details=true.
Customer's first ask to cancel order must be responded to with confirmation so call cancel order with confirmed as false. Once customer confirms call cancel order with confirmed as true. 
"""

In [106]:
from IPython.display import display_html

def add_to_chat_history(chat_history, message):
    content = message["content"]
    if not content:
        function_call = message['function_call']
        content = f"{function_call['name']}({function_call['arguments']})"
    content = content.replace("\n", "<br>")
    display_html(f"<b>{message['role']}:</b> {content}", raw=True)
    chat_history.append(message)
    return chat_history

def chat_completion(chat_history):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        engine="gpt-4",
        messages=[{"role": "system", "content": system_prompt}, *chat_history],
        temperature=0,
        functions=functions,
    )
    return response.choices[0].message

def simulate_conversation(order_api, chat_history):
    response_message = chat_completion(chat_history)
    if "function_call" not in response_message:
        chat_history = add_to_chat_history(chat_history, {
            "role": "assistant",
            "content": response_message["content"],
        })
        return chat_history

    func_name = response_message["function_call"]["name"]
    func_arguments = response_message["function_call"]["arguments"]
    chat_history = add_to_chat_history(chat_history, {
        "role": "assistant",
        "function_call": {
            "name": func_name,
            "arguments": func_arguments,
        },
        "content": None,
    })
    func = getattr(functions_code.OrderAPI, func_name)
    func_kwargs = json.loads(func_arguments)

    if func_name == "get_order_details":
        func_output = func(order_api, **func_kwargs)
        chat_history = add_to_chat_history(chat_history, {
            "role": "function",
            "name": func_name,
            "content": str(func_output),
        })
    elif func_name == "list_orders":
        func_output = func(order_api, **func_kwargs)
        chat_history = add_to_chat_history(chat_history, {
            "role": "function",
            "name": func_name,
            "content": str(func_output),
        })
        if func_kwargs["to_get_order_details"]:
            chat_history = add_to_chat_history(chat_history, {
                "role": "assistant",
                "content": "Could you please tell me which order?",
            })
    elif func_name == "cancel_order":
        if not func_kwargs["confirmed"]:
            chat_history = add_to_chat_history(chat_history, {
                "role": "assistant",
                "content": "Are you sure you want to cancel your order?",
            })
        else:
            func_output = func(order_api, **func_kwargs)
            chat_history = add_to_chat_history(chat_history, {
                "role": "function",
                "name": func_name,
                "content": str(func_output),
            })
    return chat_history

### Scenario 1: Read-only (list_orders) and State-changing (cancel_order) actions

- Customer: Where is my order? (without order_id)
- Bot: Could you please tell me which order? (list orders call)
- Customer: nike shoes one
- Bot: Your order is at Indiranagar (get order details call)
- Customer: Cancel my order
- Bot: Are you sure you want to cancel Nike shoes order? (confirmation before any state-changing action)
  - Customer: Yes
    - Bot: Your order of Nike shoes has been canceled.
  - Customer: No
    - Bot: Your order of Nike shoes will not be canceled.

In [107]:
user_inputs = [
    "where is my order?",
    "nike shoes one",
    "cancel my order",
    "yes",
]

chat_history = []
for inp in user_inputs:
    chat_history = add_to_chat_history(chat_history, {"role": "user", "content": inp})
    chat_history = simulate_conversation(order_api, chat_history)

user: where is my order?

assistant: list_orders({ "to_get_order_details": true })

function: id,item_name,status,order_date 124,Nike running shoes,OUT_FOR_DELIVERY,2023-09-11 00:00:00 123,Beige blanket,DELIVERED,2023-01-01 00:00:00 122,IKEA Table lamp,CANCELED,2022-02-28 00:00:00 121,Shampoo,DELIVERED,2022-01-01 00:00:00

assistant: Could you please tell me which order?

user: nike shoes one

...

assistant: cancel_order({ "order_id": 124, "confirmed": false })

assistant: Are you sure you want to cancel your order?

user: yes

assistant: cancel_order({ "order_id": 124, "confirmed": true })

function: Cancellation was successful.

### Scenario 2: Information retrieval

- Customer: I need invoice for my order
- Bot: Here's how to get order invoice... (retrieval from docs)